In [1]:
import torch
import rasterio
import pandas as pd
import sklearn
import numpy as np

from Model import mlp

DEVICE = 'cuda' if torch.cuda.is_available else 'cpu'
print("Device yang dipakai adalah {}".format(DEVICE))

Device yang dipakai adalah cuda


In [2]:
df = pd.read_csv('./Data/data_training.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 416 entries, 0 to 415
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Class          416 non-null    int64  
 1   crop_planet_1  416 non-null    float64
 2   crop_planet_2  416 non-null    float64
 3   crop_planet_3  416 non-null    float64
 4   crop_planet_4  416 non-null    float64
 5   crop_planet_5  416 non-null    float64
 6   crop_planet_6  416 non-null    float64
 7   crop_planet_7  416 non-null    float64
 8   crop_planet_8  416 non-null    float64
dtypes: float64(8), int64(1)
memory usage: 29.4 KB


In [3]:
from sklearn.model_selection import train_test_split

X = df.drop(columns=['Class']).values
y = df['Class'].values
y = y - 1
print(np.unique(y))

[0 1 2 3 4]


In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(X_train.shape, X_test.shape, len(np.unique(y)))

(332, 8) (84, 8) 5


In [5]:
from torch.utils.data import DataLoader, TensorDataset

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((332, 8), (84, 8), (332,), (84,))

In [6]:
num_class =len(np.unique(y))
Model_MLP = mlp.MLP(input_size=8, num_class=num_class).to(DEVICE)

In [7]:
from torch.optim import Adam

# Define training parameters
num_epochs = 500
learning_rate = 0.001
input_size = X_train_tensor.shape[1] 
num_classes = num_class
optimizer = Adam(Model_MLP.parameters(), lr=learning_rate)

# Loss function
criterion = torch.nn.CrossEntropyLoss()

print(input_size)
print(num_classes)

8
5


In [8]:
model_name = 'MLP'

# Metrics list
metrics = {
    "epoch": [],
    "train_loss": [],
    "train_accuracy": [],
    "test_loss": [],
    "test_accuracy": []
}

In [9]:
# Training loop
for epoch in range(num_epochs):
    # Set model to training mode
    Model_MLP.train()  
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    for inputs, labels in train_loader:  
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)  

        optimizer.zero_grad()
        outputs = Model_MLP(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # Calculate training accuracy
        _, predicted = torch.max(outputs.data, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    avg_train_loss = running_loss / len(train_loader)
    train_accuracy = 100 * correct_train / total_train

    # Model Evaluation (on the test set)
    Model_MLP.eval()
    correct_val = 0
    total_val = 0
    running_val_loss = 0.0
    with torch.no_grad():
        for inputs, labels in test_loader:  # Assume test_loader is already defined
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            outputs = Model_MLP(inputs)
            loss = criterion(outputs, labels)
            running_val_loss += loss.item()

            _, predicted = torch.max(outputs.data, 1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    avg_val_loss = running_val_loss / len(test_loader)
    val_accuracy = 100 * correct_val / total_val

    # Print metrics for each epoch
    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%, Test Loss: {avg_val_loss:.4f}, Test Accuracy: {val_accuracy:.2f}%")

    # Save metrics to dictionary
    metrics["epoch"].append(epoch + 1)
    metrics["train_loss"].append(avg_train_loss)
    metrics["train_accuracy"].append(train_accuracy)
    metrics["test_loss"].append(avg_val_loss)
    metrics["test_accuracy"].append(val_accuracy)

Epoch [1/500], Train Loss: 23.9468, Train Accuracy: 53.92%, Test Loss: 4.6084, Test Accuracy: 70.24%
Epoch [2/500], Train Loss: 3.9040, Train Accuracy: 72.29%, Test Loss: 3.7486, Test Accuracy: 73.81%
Epoch [3/500], Train Loss: 3.9623, Train Accuracy: 75.90%, Test Loss: 2.0185, Test Accuracy: 76.19%
Epoch [4/500], Train Loss: 6.7906, Train Accuracy: 69.58%, Test Loss: 8.0574, Test Accuracy: 50.00%
Epoch [5/500], Train Loss: 2.5295, Train Accuracy: 76.20%, Test Loss: 1.4332, Test Accuracy: 85.71%
Epoch [6/500], Train Loss: 4.1060, Train Accuracy: 72.29%, Test Loss: 1.4732, Test Accuracy: 75.00%
Epoch [7/500], Train Loss: 3.5989, Train Accuracy: 73.19%, Test Loss: 1.4636, Test Accuracy: 78.57%
Epoch [8/500], Train Loss: 1.9010, Train Accuracy: 76.81%, Test Loss: 2.0597, Test Accuracy: 83.33%
Epoch [9/500], Train Loss: 1.4419, Train Accuracy: 81.02%, Test Loss: 1.9717, Test Accuracy: 85.71%
Epoch [10/500], Train Loss: 1.3858, Train Accuracy: 82.23%, Test Loss: 1.1262, Test Accuracy: 84.52

In [10]:
# Save model kalau mau digunakan lagi di lain waktu
torch.save(Model_MLP.state_dict(), f"{model_name}_best.pth")
print(f"{model_name} saved as {model_name}_best.pth\n")

# Save the metrics to a CSV file
df = pd.DataFrame(metrics)
df.to_csv(f"{model_name}_metrics.csv", index=False)
print(f"Metrics for {model_name} saved as {model_name}_metrics.csv\n")

MLP saved as MLP_best.pth

Metrics for MLP saved as MLP_metrics.csv

